In [1]:
from bolib import Parameter
from bolib.Dimension import DiscreteDimension, NumericDimension
from bolib.Bo import Bo
from bolib.ComputeSpace import ComputeList, ComputeSpace
import random

from bolib.Parameter import ParamList


c:\Users\phili\Documents\workspace\botorch\botorch_contrast\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:



gender      = DiscreteDimension(elements=["male", "female", "diverse"] ,name = "gender")
age         = NumericDimension(min=18, max=70, name="age")
height      = NumericDimension(min=160, max=190, name="height")
haircolor   = DiscreteDimension(elements=["blonde", "brown", "black", "grey", "red", "blue", "pink", "colorful"] ,name = "haircolor")
hairstyle   = DiscreteDimension(elements=["curly", "straight"] ,name = "hairstyle")
skintone    = DiscreteDimension(elements=["white", "black", "latino", "native american", "asian"] ,name = "skintone")
bodytype    = DiscreteDimension(elements=["slim", "athletic", "average", "curvy"], name = "body type")    

RANK_MAX = 5
RANK_MIN = 1

ranking = [ NumericDimension(min=1, max=5, name="Rating") ]
dimensions = [gender, age, height, haircolor, hairstyle, skintone, bodytype]
compSpace = ComputeSpace(x = dimensions, y = ranking)


In [8]:
from bolib.Float01 import Float01


optimizer = Bo(compSpace)

normalized_guess = [random.random() for _ in dimensions]
values = compSpace.denormalize(normalized_guess)
target = ParamList([gender.new('male'), age.new(36), height.new(176), haircolor.new("black"),hairstyle.new("straight"), skintone.new("white"),bodytype.new("average")])
min_guess = ParamList([gender.new('female'), age.new(70), height.new(160), haircolor.new("colorful"),hairstyle.new("curly"), skintone.new("black"),bodytype.new("slim")])




def get_rating(l: ParamList):
    p0 = 1 if l[0].value == "male" else 0
    
    p1 = 1-l[1].normalized_distance(target[1])
    p2 = 1-l[2].normalized_distance(target[2])
    
    
    p3 = 1 if l[3].value == "black" else 0
    p4 = 1 if l[4].value == "straight" else 0
    p5 = 1 if l[5].value == "white" else 0
    p6 = 1 if l[6].value == "average" else 0
    
    ret = [p0, p1, p2, p3, p4, p5, p6]
    print(ret)
    return ((sum(ret)/len(ret)))*RANK_MAX

#height.new(176).normalized

get_rating(min_guess)



[0, 0.0, 0.0, 0, 0, 0, 0]


0.0

In [ ]:



print("Welcome to the character guessing game.")
print("The idea is that the computer tells you a descroption of yourself and you have to indicate how accurate the description is.")
print(f"You tell the computer how good the guess is.")

print("After some iterations, the description should fit")

print("")
print(f"Please make a single rating on a scale from {RANK_MIN} to {RANK_MAX}, how well the description fits you")
print("")
print(f"{RANK_MIN} = fits not at all")
print(f"{RANK_MAX} = fits very well")
print("")
input("enter to proceed")

print("Great! Now let's start")
tries = 0
while True:
    print("Are you this person?")
    guess = compSpace.denormalize(normalized_guess)
    print(guess.to_dataframe()[['name', "value"]])
    
    """
    feedback = 0
    try:
        feedback = float(input(f"Enter rating [{RANK_MIN} = fits not at all, {RANK_MAX} = fits very well]: "))        
    except:
        break
    """
    rating = round(get_rating(guess))
    print(f"rating: {rating}")
    compSpace.add_value(guess, rating)
    
    normalized_guess = optimizer.infer()        
    
    if rating == RANK_MAX:
        print(f"Great! I guessed it in {tries} tries!")
        break        
    
    print("\n")
    input("next?")


    tries += 1